# Analyse av dokumentsplitting for NKS kunnskapsbasen

Denne notatboken utforsker hvordan man best kan splitte `LangChain` dokumentene
som blir opprettet fra NKS sin kunnskapsbase. En god splitt er viktig fordi det
gir bedre treffsikkerhet for KBS systemet.

In [ ]:
from rich.console import Console

console = Console()

## Last inn dokumenter BigQuery

In [ ]:
from nks_kbs_analyse.knowledgebase import load

docs = list(load())
console.print(f"Lastet inn {len(docs)} dokumenter")

## Default splitt i dag

In [ ]:
from nks_kbs_analyse.knowledgebase import split_documents

# MERK: 1500 kommer fra analyse utført av Beate som viste at 97% av alle
# paragrafer er mindre eller lik 1500 tegn
split_docs = split_documents(docs, chunk_size=1500, overlap=100)
console.print(f"Etter splitt er det {len(split_docs)} dokumenter")

Utfordringen med dagens splitt er at den resulterer i en del mindre dokumenter
som ikke gir noe verdi og i mange tilfeller fjernes det potensielt viktig
informasjon.

In [ ]:
small_docs = [doc.page_content for doc in split_docs if len(doc.page_content) <= 50]
console.print(f"Antall små dokumenter: {len(small_docs)}")
console.print(small_docs)

## Alternative måter å splitte

### Endre parameter på rekursiv splitter

In [ ]:
from langchain_text_splitters import Language, RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter.from_language(
    language=Language.MARKDOWN, chunk_size=4000, chunk_overlap=200
)
split_docs = text_splitter.transform_documents(docs)
console.print(f"Etter splitt er det {len(split_docs)} dokumenter")

In [ ]:
small_docs = [doc.page_content for doc in split_docs if len(doc.page_content) <= 50]
console.print(f"Antall små dokumenter: {len(small_docs)}")
console.print(small_docs)

## Prøve å vaske dokumenter før splitting

In [ ]:
from langchain_core.documents import Document


def clean_doc(doc: Document) -> Document:
    """Prøv å rense dokument med enkle regex-er."""
    import re

    new_content = doc.page_content
    # Fjerne for mange newline etter overskrift
    new_content = re.sub(r"(#{1,6}.*\n)\n+", r"\1", new_content)
    # Fjerne newline hvis det er 3 eller flere newline på rad
    new_content = re.sub(r"(\n\n)\s+", r"\1", new_content)
    # Fjerne tomme markdown overskrifter
    new_content = re.sub(r"^#{1,6}\s*\n", "", new_content, flags=re.MULTILINE)
    return Document(page_content=new_content, metadata=doc.metadata)

In [ ]:
clean_docs = [clean_doc(doc) for doc in docs]

In [ ]:
split_clean = split_documents(clean_docs, chunk_size=1500, overlap=100)
console.print(f"Etter splitt er det {len(split_clean)} dokumenter")

In [ ]:
small_docs = [doc.page_content for doc in split_clean if len(doc.page_content) <= 50]
console.print(f"Antall små dokumenter: {len(small_docs)}")
console.print(small_docs)